In [1]:
from qiskit import *
from qiskit.visualization import plot_histogram
import numpy as np

def shors_algorithm(N, a):
    # Step 1: Choose a random integer 'a' such that 1 < a < N
    # Note: The choice of 'a' can affect the success probability of the algorithm
    
    # Step 2: Create a quantum circuit with 2n+3 qubits
    n = N.bit_length()  # Number of qubits needed to represent N
    qc = QuantumCircuit(2 * n + 3, n)
    
    # Step 3: Initialize the qubits
    qc.h(range(n))  # Apply Hadamard gates to the first n qubits
    
    # Step 4: Modular exponentiation
    for q in range(n):
        qc.x(n + q)  # Set the second register to |1>
        qc.append(c_amod15(a, 2**q), 
                 [q] + list(range(n, 2 * n + 3)))  # Controlled modular exponentiation
    
    # Step 5: Quantum Fourier Transform
    qc.append(qft_dagger(n), range(n))  # Apply the inverse QFT to the first n qubits
    
    # Step 6: Measure the first n qubits
    qc.measure(range(n), range(n))
    
    # Step 7: Execute the circuit on a simulator
    aer_sim = Aer.get_backend('aer_simulator')
    # Setting the number of shots to a higher value increases the success probability
    t_qc = transpile(qc, aer_sim)
    qobj = assemble(t_qc, shots=1)
    result = aer_sim.run(qobj).result()
    
    # Step 8: Analyze the results to find a non-trivial factor
    counts = result.get_counts()
    for measured_value in counts:
        measured_int = int(measured_value, 2)
        
        # Apply continued fractions to find the possible factors
        frac = continued_fraction(measured_int, 2**n)
        if frac.denominator % 2 != 0:
            # If the denominator is odd, try a different random 'a'
            return None
        x = int((a**(frac.denominator//2)) % N)
        if (x + 1) % N != 0:
            return x, N // (x + 1)

    return None
def c_amod15(a, power):
    U = QuantumCircuit(4)
    for iteration in range(power):
        U.append(c_amod15_gate(a, iteration), 
                 [0, 1, 2, 3])
    U = U.to_gate()
    U.name = "a^%d mod 15" % a
    c_U = U.control()
    return c_U

# Function to compute modular exponentiation
def c_amod15_gate(a, power):
    U = QuantumCircuit(4)
    for q in range(4):
        U.x(q)
    U = U.to_gate()
    U.name = "X"
    
    power_binary = format(power, '02b')[::-1]
    for i in range(len(power_binary)):
        if power_binary[i] == '1':
            U = U.compose(c_amod15_power(a, 2**i), qubits=[0, 1, 2, 3])

    U = U.to_gate()
    U.name = "a^%d mod 15" % a
    return U
def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n - 1 - qubit)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j - m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

# Function to compute modular exponentiation for a specific power of 2
def c_amod15_power(a, power):
    U = QuantumCircuit(4)
    for iteration in range(power):
        U.append(c_amod15_gate_helper(a, iteration), [0, 1, 2, 3])
    return U.to_gate()

# Helper function for the inner loop of modular exponentiation
def c_amod15_gate_helper(a, power):
    U = QuantumCircuit(4)
    for q in range(4):
        U.x(q)**((a**power) % 15)
    return U.to_gate()

# Now, rerun the Shor's algorithm with the corrected functions

result = shors_algorithm(15, 2)

if result:
    print(f"Non-trivial factors of {N}: {result}")
else:
    print(f"Failed to find non-trivial factors for {N} with the chosen 'a'. Try a different 'a'.")


AttributeError: 'Gate' object has no attribute 'to_gate'